# Context
Problem: The bigger the data set, the more likely "rare" events are to appear<BR>
Solution: normalize by likelihood of the event being random <BR>
https://www.kdnuggets.com/2016/07/big-data-bible-codes-bonferroni.html    
    
Collecting data, storing data, and analyzing data has a cost.<BR>
The bigger the data set, the higher the cost, the more experiments you are going to run to get value.<BR>
Problem: By <a href="https://en.wikipedia.org/wiki/Multiple_comparisons_problem">increasing the number of tests</a>, you're more likely to find what you're looking for due to chance.<BR>
Solution: https://en.wikipedia.org/wiki/Bonferroni_correction<BR>
https://www.stat.berkeley.edu/~mgoldman/Section0402.pdf
    


# In this notebook

We'll generate random data and look for patterns that are rare.

As the size of the random data is increased, we find more of these "rare" outcomes. 

We can calculate whether this increase is expected or not.

In [1]:
import random
import string
import time
import matplotlib.pyplot as plt

use the "string" module to get a list of characters in the alphabet

In [2]:
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

Before we generate a string of size N,

In [3]:
N=10

In [4]:
26**N

141167095653376

to generate the 10 character random string, we could write a loop:

In [5]:
start=time.time()
this_str=[] # use a list to store the characters
for digit_indx in range(N):
    a_char=random.choice(string.ascii_uppercase) 
    this_str.append(a_char)

print(''.join(this_str)) # combine the characters in the list into a string with no separators
print('elapsed time:',round(time.time()-start,5),'seconds') # how long did that take?

IXBGZFQNLC
elapsed time: 0.01412 seconds


A much shorter way to write the same loop is to use a list comprehension.

For example,

In [6]:
['a' for digit_indx in range(3)]

['a', 'a', 'a']

Instead of `'a'`, use the `random.choice()` function

Rather than naming the loop index variable, we can use an unnamed variable

In [7]:
[random.choice(string.ascii_uppercase) for _ in range(N)]

['A', 'B', 'W', 'W', 'R', 'G', 'D', 'Z', 'P', 'E']

Lastly, we don't actually need a list, we are just going to combing it into a string:

In [8]:
start=time.time()
my_str=''.join(random.choice(string.ascii_uppercase) for _ in range(N)) # select a random character N times
print('elapsed time:',round(time.time()-start,5),'seconds') # how long did that take?

elapsed time: 0.00087 seconds


In [9]:
my_str

'JDGPOKYXLF'

Using a list comprehension is slightly faster than the for loop and more concise, but those are not usually our objectives

Find a "pattern" in random data:

In [10]:
def one_match(str_to_match,
              number_of_random_strings_to_inspect,
              str_len,prt_bool):
    """
    inputs: 
       * a string to find within a randomly generated string
       * the number of tests to run
       * a boolean to either print or not print output
    outputs:
       * the number of times the input string appears in a random string
    """
    # count the number of matches for the input and the random strings
    number_of_results=0
    for test_indx in range(number_of_random_strings_to_inspect): # ct = number of test to run
        # generate a random string
        my_str=''.join(random.choice(string.ascii_uppercase) for _ in range(str_len))
        # test to determine whether the input string is in the random string
        if (str_to_match in my_str):
            # if a match is found, increment the success counter
            number_of_results+=1
            if prt_bool:
                print(my_str)
    if prt_bool:
        print('number of results:',number_of_results)
    return(number_of_results)

In [11]:
one_match('A',10,N,True)

number of results: 0


0

The probability of getting "A" in the first position of the 10 character string is 1/26.<BR>
The probability of getting "A" in the second position of the 10 character string is 1/26.<BR>
The probability of getting "A" in the third position of the 10 character string is 1/26.<BR>
Therefore, the probability of getting "A" in any of 10 positions is

In [12]:
(1/26)+(1/26)+(1/26)+(1/26)+(1/26)+(1/26)+(1/26)+(1/26)+(1/26)+(1/26)

0.3846153846153846

In [13]:
one_match('AAA',10,N,True)

number of results: 0


0

Therfore, the chance of getting "AAA" in the first three character is 

In [24]:
(1/26)*(1/26)*(1/26)

5.689576695493856e-05

I don't care where "AAA" shows up in the 10 character string, so the outcome for any of the 3 adjacent letters is

In [25]:
prob_find_AAA=(
# AAA_______
((1/26)*(1/26)*(1/26))+
# _AAA______
((1/26)*(1/26)*(1/26))+
# __AAA_____
((1/26)*(1/26)*(1/26))+
# ___AAA____
((1/26)*(1/26)*(1/26))+
# ____AAA___
((1/26)*(1/26)*(1/26))+
# _____AAA__
((1/26)*(1/26)*(1/26))+
# ______AAA_
((1/26)*(1/26)*(1/26))+
# _______AAA
((1/26)*(1/26)*(1/26))
)

prob_find_AAA

0.00045516613563950854

In [26]:
number_of_tests_to_run=10000

If we run 10,000 tests, what's the average number of matches?

In [27]:
round(number_of_tests_to_run*prob_find_AAA,2)

4.55

In [29]:
one_match('AAA',number_of_tests_to_run,N,True)

TAAAERMJMF
JNRFAAAFUE
DGAAAJEHOL
AAAEGBDEKJ
number of results: 4


4

The more "patterns" we look for, the more successes we will find. 

Solution: normalize your "successes" count by the number of patterns you're looking for

In [30]:
def two_matches(str1_to_match,
                str2_to_match,
                number_of_random_strings_to_inspect,
                str_len,
                prt_bool):
    number_of_results=0
    for test_indx in range(number_of_random_strings_to_inspect):
        my_str=''.join(random.choice(string.ascii_uppercase) for _ in range(str_len))
        if (str1_to_match in my_str) or (str2_to_match in my_str):
            number_of_results+=1
            if prt_bool:
                print(my_str)
    if prt_bool:
        print('number of results:',number_of_results)
    return(number_of_results)

In [31]:
rst_ct=two_matches('AAA','BBB',number_of_tests_to_run,N,True)

MBBBIMRHIX
QUAAATCPIY
QSCNAAAPXQ
EADBBBXHIO
BKJZBBBFRF
YBBBAOKYQX
XXQUZZUAAA
BQBBBUTZUY
KBBBCQJJYO
number of results: 9


That's higher than when we searched for only 'AAA'

Though we found more results, we need to normalize the count by the number of searches we made:

In [32]:
rst_ct/2

4.5

which is close to the original expection

In [33]:
round(number_of_tests_to_run*prob_find_AAA,2)

4.55

<!-- # the rest of this content is irrelevant -->

<!-- Caveat: the following takes a long time (about 2 minutes) to run -->

<!-- 
start=time.time()
list_of_results=[]
for indx in range(1000):
    rst_ct=two_matches('AAA','BBB',number_of_tests_to_run,N,False)
    list_of_results.append(rst_ct)
print(time.time()-start)
-->

<!-- 
_=plt.hist(list_of_results)
_=plt.xlabel('number of matches',fontsize=14)
_=plt.ylabel('count',fontsize=14)
-->

<BR>
    <BR>
        <BR>
            <BR>
                <BR>
                    <BR>
<!-- As we scale up the amount of data we search, we find more successes -->